In [1]:
'''
Efn-B4 Model
384x384 image data (Triple Stratified Data)
40 epochs
3 Folds
Early Stopping
Tabular Data + Image Data
Focal Loss
Using External Data
'''

'\nEfn-B4 Model\n384x384 image data (Triple Stratified Data)\n40 epochs\n3 Folds\nEarly Stopping\nTabular Data + Image Data\nFocal Loss\nUsing External Data\n'

In [2]:
Kaggle = False
Colab = !Kaggle

In [3]:
!pip install tensorflow~=2.2.0 tensorflow_gcs_config~=2.2.0
import tensorflow as tf
import requests
import os
resp = requests.post("http://{}:8475/requestversion/{}".format(os.environ["COLAB_TPU_ADDR"].split(":")[0], tf.__version__))
if resp.status_code != 200:
  print("Failed to switch the TPU to TF {}".format(version))

In [4]:
import os, sys
from pathlib import Path

if Colab:
    from google.colab import drive
    drive.mount('/content/drive')

    path = "/content/drive/My Drive"
    os.chdir(path)
    os.listdir(path)
else:
    from kaggle_datasets import KaggleDatasets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
if Kaggle:
    BASEPATH = "../input/siim-isic-melanoma-classification"
    outdir = '.'
else:
    PATH = 'siim/'
    BASEPATH = PATH + 'siim-isic-melanoma-classification'
    outdir = Path(PATH+'res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    outdir = Path(PATH+'res/efficientnet-res')
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    MODELNAME = "EF5-512"
    VERSION = '{}'.format(MODELNAME)
    outdir = os.path.join(outdir, VERSION)
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    # from datetime import datetime, timedelta
    # dateTimeObj = datetime.now()
    # timestampStr = dateTimeObj.strftime("%d-%b-%Y-%H")
    timestampStr = 'all-kfold-hairaug-456-norm-meta-dense'
    outdir = os.path.join(outdir, timestampStr)
    if not os.path.exists(outdir):
        os.mkdir(outdir)   

In [6]:
!pip install -q efficientnet >> /dev/null

import os
import re
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import KFold, StratifiedKFold
import tensorflow as tf
# from kaggle_datasets import KaggleDatasets
import efficientnet.tfkeras as efn
import dill
from tensorflow.keras import backend as K
import tensorflow_addons as tfa

In [7]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.71.222.18:8470
INFO:tensorflow:Initializing the TPU system: grpc://10.71.222.18:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.71.222.18:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [8]:
## melanoma dataset
gcs_path = []
gcs_path.append('gs://kds-2b28e4519749435c2fb543681ca66aaee6159d9fcc906d5b26f8916e') #192
gcs_path.append('gs://kds-f8af4bf2f6c765dbd327683a170e364332a1750658e910153e98d0d5') #256
gcs_path.append('gs://kds-71b679af6661316328cfa122f5342b4a84fe0c7c1135d809fbee8ca4') #384
gcs_path.append('gs://kds-458e4ec4a30e5d927b21efd0633fb2c0a091245fe8db3c415fde5da6') #512
gcs_path.append('gs://kds-cfb16ed5f55adb5ab35a75a2fe74c3dc11a4b869dc8cfb3d9b1759e1') #768

## isic dataset
gcs_path2 = []
gcs_path2.append('gs://kds-c9c10ac29c6818441da475638226dfcf3d12806980a23f179a21d826') #192-2018
gcs_path2.append('gs://kds-436e76b7e255e4499d5d11da9d7ab87646e271e08ec1d546f310ec00') #256-2018
gcs_path2.append('gs://kds-6f9d2143e7fe97eccac0219c1aacb107a234b440670f250683ba40ee') #384-2018
gcs_path2.append('gs://kds-203bea6cc14c2866536ac303b09a6e77a6b3d0a371e96b680836433c') #512-2018
gcs_path2.append('gs://kds-79d9c56dcae7978f98a6bbe8318ac6866ba4778fc95c5871fcfd0f03') #768-2019 

gcs_path_dict = {192:0,256:1,384:2,512:3,768:4}

gcs_path_hairs = 'gs://kds-0f1845648605e1d5c6a531816acb4e993458e02c32d8f4d5bc6bf4cf'

In [9]:
# Roman's images of hairs
hair_images=tf.io.gfile.glob(gcs_path_hairs + '/*.png')

In [10]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE
n_max = 10
# Data access
img_size = 456
# IMG_SIZES = [img_size,img_size,img_size,img_size,img_size]
img_read_size = 512
# IMG_READ_SIZES = [img_read_size,img_read_size,img_read_size,img_read_size,img_read_size]
img_crop_size = 452
if Kaggle:
    GCS_PATH = KaggleDatasets().get_gcs_path('melanoma-{}x{}'.format(img_read_size, img_read_size))
    GCS_PATH2 = KaggleDatasets().get_gcs_path('isic2019-{}x{}'.format(img_read_size, img_read_size))
else:    
    GCS_PATH = gcs_path[gcs_path_dict[img_read_size]]
    GCS_PATH2 = gcs_path2[gcs_path_dict[img_read_size]]

# Configuration
EPOCHS = 40
BATCH_SIZE = 8 * strategy.num_replicas_in_sync
AUG_BATCH = BATCH_SIZE

# Seed
SEED = 42
# Learning rate
LR = 0.0003

# training filenames directory
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train*.tfrec')
TRAINING_FILENAMES_OLD_DATA = tf.io.gfile.glob(GCS_PATH2 + '/train%.2i*.tfrec'%(2*x) for x in range(15))

# test filenames directory
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test*.tfrec')
# submission file
SUB = pd.read_csv('siim/siim-isic-melanoma-classification/sample_submission.csv')

In [11]:
len(TRAINING_FILENAMES)

15

In [12]:
TRAINING_FILENAMES += TRAINING_FILENAMES_OLD_DATA

In [13]:
len(TRAINING_FILENAMES)

30

In [14]:
# Taken from following
#https://github.com/yu4u/cutout-random-erasing/blob/master/random_eraser.py
'''
Parameters
Parameters are fully configurable as:

get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3,
                  v_l=0, v_h=255, pixel_level=False)
p        : the probability that random erasing is performed
s_l, s_h : minimum / maximum proportion of erased area against input image
r_1, r_2 : minimum / maximum aspect ratio of erased area
v_l, v_h : minimum / maximum value for erased area
pixel_level : pixel-level randomization for erased area
''' 
def get_random_eraser(p=0.6, s_l=0.02, s_h=0.4, r_1=0.2, r_2=1/0.2, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        img_h, img_w, img_c = input_img.shape
        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            c = np.random.uniform(v_l, v_h, (h, w, img_c))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w, :].assign(tf.Variable(c))

        return input_img

    return eraser

In [15]:
cutout_aug = get_random_eraser(s_h=0.2)

In [16]:
def random_erasing(img, probability = 0.5, sl = 0.02, sh = 0.2, r1 = 0.1):
    '''
    img is a 3-D variable (ex: tf.Variable(image, validate_shape=False) ) and  HWC order
    '''
    # HWC order
    height = tf.shape(img)[0]
    width = tf.shape(img)[1]
    channel = tf.shape(img)[2]
    area = tf.cast(width*height, tf.float32)

    erase_area_low_bound = tf.cast(tf.round(tf.sqrt(sl * area * r1)), tf.int32)
    erase_area_up_bound = tf.cast(tf.round(tf.sqrt((sh * area) / r1)), tf.int32)
    h_upper_bound = tf.minimum(erase_area_up_bound, height)
    w_upper_bound = tf.minimum(erase_area_up_bound, width)

    h = tf.random.uniform([], erase_area_low_bound, h_upper_bound, tf.int32)
    w = tf.random.uniform([], erase_area_low_bound, w_upper_bound, tf.int32)

    x1 = tf.random.uniform([], 0, height+1 - h, tf.int32)
    y1 = tf.random.uniform([], 0, width+1 - w, tf.int32)

    erase_area = tf.cast(tf.random.uniform([h, w, channel], 0, 255, tf.int32), tf.uint8)

    erasing_img = img[x1:x1+h, y1:y1+w, :].assign(erase_area)

    return tf.cond(tf.random.uniform([], 0, 1) > probability, lambda: img, lambda: erasing_img)

# CUTOUT Aug

In [17]:
def transform_cut(image, inv_mat, image_shape):

    h, w, c = image_shape
    cx, cy = w//2, h//2

    new_xs = tf.repeat( tf.range(-cx, cx, 1), h)
    new_ys = tf.tile( tf.range(-cy, cy, 1), [w])
    new_zs = tf.ones([h*w], dtype=tf.int32)

    old_coords = tf.matmul(inv_mat, tf.cast(tf.stack([new_xs, new_ys, new_zs]), tf.float32))
    old_coords_x, old_coords_y = tf.round(old_coords[0, :] + w//2), tf.round(old_coords[1, :] + h//2)

    clip_mask_x = tf.logical_or(old_coords_x<0, old_coords_x>w-1)
    clip_mask_y = tf.logical_or(old_coords_y<0, old_coords_y>h-1)
    clip_mask = tf.logical_or(clip_mask_x, clip_mask_y)

    old_coords_x = tf.boolean_mask(old_coords_x, tf.logical_not(clip_mask))
    old_coords_y = tf.boolean_mask(old_coords_y, tf.logical_not(clip_mask))
    new_coords_x = tf.boolean_mask(new_xs+cx, tf.logical_not(clip_mask))
    new_coords_y = tf.boolean_mask(new_ys+cy, tf.logical_not(clip_mask))

    old_coords = tf.cast(tf.stack([old_coords_y, old_coords_x]), tf.int32)
    new_coords = tf.cast(tf.stack([new_coords_y, new_coords_x]), tf.int64)
    rotated_image_values = tf.gather_nd(image, tf.transpose(old_coords))
    rotated_image_channel = list()
    for i in range(c):
        vals = rotated_image_values[:,i]
        sparse_channel = tf.SparseTensor(tf.transpose(new_coords), vals, [h, w])
        rotated_image_channel.append(tf.sparse.to_dense(sparse_channel, default_value=0, validate_indices=False))

    return tf.transpose(tf.stack(rotated_image_channel), [1,2,0])

def random_rotate(image, angle, image_shape):

    def get_rotation_mat_inv(angle):
          #transform to radian
        angle = math.pi * angle / 180

        cos_val = tf.math.cos(angle)
        sin_val = tf.math.sin(angle)
        one = tf.constant([1], tf.float32)
        zero = tf.constant([0], tf.float32)

        rot_mat_inv = tf.concat([cos_val, sin_val, zero,
                                     -sin_val, cos_val, zero,
                                     zero, zero, one], axis=0)
        rot_mat_inv = tf.reshape(rot_mat_inv, [3,3])

        return rot_mat_inv
    angle = float(angle) * tf.random.normal([1],dtype='float32')
    rot_mat_inv = get_rotation_mat_inv(angle)
    return transform_cut(image, rot_mat_inv, image_shape)


def GridMask(image_height, image_width, d1, d2, rotate_angle=1, ratio=0.2):

    h, w = image_height, image_width
    hh = int(np.ceil(np.sqrt(h*h+w*w)))
    hh = hh+1 if hh%2==1 else hh
    d = tf.random.uniform(shape=[], minval=d1, maxval=d2, dtype=tf.int32)
    l = tf.cast(tf.cast(d,tf.float32)*ratio+0.5, tf.int32)

    st_h = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)
    st_w = tf.random.uniform(shape=[], minval=0, maxval=d, dtype=tf.int32)

    y_ranges = tf.range(-1 * d + st_h, -1 * d + st_h + l)
    x_ranges = tf.range(-1 * d + st_w, -1 * d + st_w + l)

    for i in range(0, hh//d+1):
        s1 = i * d + st_h
        s2 = i * d + st_w
        y_ranges = tf.concat([y_ranges, tf.range(s1,s1+l)], axis=0)
        x_ranges = tf.concat([x_ranges, tf.range(s2,s2+l)], axis=0)

    x_clip_mask = tf.logical_or(x_ranges <0 , x_ranges > hh-1)
    y_clip_mask = tf.logical_or(y_ranges <0 , y_ranges > hh-1)
    clip_mask = tf.logical_or(x_clip_mask, y_clip_mask)

    x_ranges = tf.boolean_mask(x_ranges, tf.logical_not(clip_mask))
    y_ranges = tf.boolean_mask(y_ranges, tf.logical_not(clip_mask))

    hh_ranges = tf.tile(tf.range(0,hh), [tf.cast(tf.reduce_sum(tf.ones_like(x_ranges)), tf.int32)])
    x_ranges = tf.repeat(x_ranges, hh)
    y_ranges = tf.repeat(y_ranges, hh)

    y_hh_indices = tf.transpose(tf.stack([y_ranges, hh_ranges]))
    x_hh_indices = tf.transpose(tf.stack([hh_ranges, x_ranges]))

    y_mask_sparse = tf.SparseTensor(tf.cast(y_hh_indices, tf.int64),  tf.zeros_like(y_ranges), [hh, hh])
    y_mask = tf.sparse.to_dense(y_mask_sparse, 1, False)

    x_mask_sparse = tf.SparseTensor(tf.cast(x_hh_indices, tf.int64), tf.zeros_like(x_ranges), [hh, hh])
    x_mask = tf.sparse.to_dense(x_mask_sparse, 1, False)

    mask = tf.expand_dims( tf.clip_by_value(x_mask + y_mask, 0, 1), axis=-1)

    #mask = random_rotate(mask, rotate_angle, [hh, hh, 1])
    mask = tf.image.crop_to_bounding_box(mask, (hh-h)//2, (hh-w)//2, image_height, image_width)

    return mask

def apply_grid_mask(image, image_shape):
    mask = GridMask(image_shape[0],
                    image_shape[1],
                    50,100,
                    0.2)
    
    if image_shape[-1] == 3:
        mask = tf.concat([mask, mask, mask], axis=-1)

    return image * tf.cast(mask, tf.float32)

def random_cuout_aug(image):
    if tf.random.uniform(shape=[], minval=0.0, maxval=1.0) >=0.5:
        image = apply_grid_mask(image, (img_size, img_size, 3))
    return tf.cast(image, tf.float32)

In [18]:
hair_images_tf=tf.convert_to_tensor(hair_images)
scale=tf.cast(img_size/256, dtype=tf.int32)

In [19]:
def hair_aug(image):
    # Copy the input image, so it won't be changed
    img=tf.identity(image) 
    # Randomly choose the number of hairs to augment (up to n_max)
    n_hairs = tf.random.uniform(shape=[], maxval=tf.constant(n_max)+1, 
                                dtype=tf.int32)
    
    im_height=tf.shape(img)[1]
    im_width=tf.shape(img)[0]
    
    if n_hairs == 0:
        return image

    for _ in tf.range(n_hairs):

        # Read a random hair image
        i=tf.random.uniform(shape=[], maxval=tf.shape(hair_images_tf)[0], 
                            dtype=tf.int32)
        fname=hair_images_tf[i]

        bits = tf.io.read_file(fname)
        hair = tf.image.decode_jpeg(bits)
        
        # Rescale the hair image to the right size (256 -- original size)
        new_width=scale*tf.shape(hair)[1]
        new_height=scale*tf.shape(hair)[0]
        hair = tf.image.resize(hair, [new_height, new_width])

        
        # Random flips of the hair image
        hair = tf.image.random_flip_left_right(hair)
        hair = tf.image.random_flip_up_down(hair)
        # Random number of 90 degree rotations
        n_rot=tf.random.uniform(shape=[], maxval=4,
                                dtype=tf.int32)
        hair = tf.image.rot90(hair, k=n_rot)
        
        h_height=tf.shape(hair)[0]
        h_width=tf.shape(hair)[1]
        
        roi_h0 = tf.random.uniform(shape=[], maxval=im_height - h_height + 1, 
                                    dtype=tf.int32)
        roi_w0 = tf.random.uniform(shape=[], maxval=im_width - h_width + 1, 
                                    dtype=tf.int32)


        roi = img[roi_h0:(roi_h0 + h_height), roi_w0:(roi_w0 + h_width)]  

        # Convert the hair image to grayscale 
        # (slice to remove the trainsparency channel)
        hair2gray = tf.image.rgb_to_grayscale(hair[:, :, :3])

        mask=hair2gray>10

        img_bg = tf.multiply(roi, tf.cast(tf.image.grayscale_to_rgb(~mask),
                                          dtype=tf.float32))
        hair_fg = tf.multiply(tf.cast(hair[:, :, :3], dtype=tf.int32),
                              tf.cast(tf.image.grayscale_to_rgb(mask), 
                                      dtype=tf.int32
                                      )
                             )

        dst = tf.add(img_bg, tf.cast(hair_fg, dtype=tf.float32)/255)

        paddings = tf.stack([
            [roi_h0, im_height-(roi_h0 + h_height)], 
            [roi_w0, im_width-(roi_w0 + h_width)],
            [0, 0]
        ])

        # Pad dst with zeros to make it the same shape as image.
        dst_padded=tf.pad(dst, paddings, "CONSTANT")
        # Create a boolean mask with zeros at the pixels of
        # the augmentation segment and ones everywhere else
        mask_img=tf.pad(tf.ones_like(dst), paddings, "CONSTANT")
        mask_img=~tf.cast(mask_img, dtype=tf.bool)
        # Make a hole in the original image at the location
        # of the augmentation segment
        img_hole=tf.multiply(img, tf.cast(mask_img, dtype=tf.float32))
        # Inserting the augmentation segment in place of the hole
        img=tf.add(img_hole, dst_padded)
        
    return img

In [20]:
def norm_image(data, label):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    data['inp1'] -= mean
    data['inp1'] /= std
    # mean_tensor = K.constant(-np.array(mean), tf.float32)
    # std_tensor = K.constant(np.array(std), tf.float32)
    # img = K.bias_add(img, mean_tensor, "channels_last")
    # img /= std_tensor
    return data, label

In [21]:
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return K.dot(K.dot(rotation_matrix, shear_matrix), K.dot(zoom_matrix, shift_matrix))

def transform(image, label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = img_size
    XDIM = DIM%2 #fix for size 331
    
    if 0.5 > tf.random.uniform([1], minval = 0, maxval = 1):
        rot = 15. * tf.random.normal([1],dtype='float32')
    else:
        rot = 180. * tf.random.normal([1],dtype='float32')
    shr = 5. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32')/10.
    h_shift = 16. * tf.random.normal([1],dtype='float32') 
    w_shift = 16. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image['inp1'],tf.transpose(idx3))
        
    return {'inp1': tf.reshape(d,[DIM,DIM,3]), 'inp2': image['inp2']}, label

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# function to decode our images (normalize and reshape)
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    # convert image to floats in [0, 1] range
    image = tf.cast(image, tf.float32) / 255.0 
    # explicit size needed for TPU
    image = tf.reshape(image, [img_read_size, img_read_size, 3])

    image = tf.image.resize(image, [img_size, img_size])
    return image

# this function parse our images and also get the target variable
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "target": tf.io.FixedLenFeature([], tf.int64),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
        
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['target'], tf.int32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, label, data)
    return image, label, data

# this function parse our image and also get our image_name (id) to perform predictions
def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        # tf.string means bytestring
        "image": tf.io.FixedLenFeature([], tf.string), 
        # shape [] means single element
        "image_name": tf.io.FixedLenFeature([], tf.string),
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    # returns a dataset of (image, key, data)
    return image, image_name, data
    
def load_dataset(filenames, labeled = True, ordered = False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # Diregarding data order. Order does not matter since we will be shuffling the data anyway
    
    ignore_order = tf.data.Options()
    if not ordered:
        # disable order, increase speed
        ignore_order.experimental_deterministic = False 
        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # use data as soon as it streams in, rather than in its original order
    dataset = dataset.with_options(ignore_order)
    # returns a dataset of (image, label) pairs if labeled = True or (image, id) pair if labeld = False
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls = AUTO) 
    return dataset

# function for training and validation dataset
def setup_input1(image, label, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, label

# function for the test set
def setup_input2(image, image_name, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name

# function for the validation (image name)
def setup_input3(image, image_name, target, data):
    
    # get anatom site general challenge vectors
    anatom = [tf.cast(data['anatom_site_general_challenge'][i], dtype = tf.float32) for i in range(7)]
    
    tab_data = [tf.cast(data[tfeat], dtype = tf.float32) for tfeat in ['age_approx', 'sex']]
    
    tabular = tf.stack(tab_data + anatom)
    
    return {'inp1': image, 'inp2':  tabular}, image_name, target

def data_augment(data, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement 
    # in the next function (below), this happens essentially for free on TPU. 
    # Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    data['inp1'] = hair_aug(data['inp1'])
    data['inp1'] = tf.image.random_flip_left_right(data['inp1'])
    data['inp1'] = tf.image.random_flip_up_down(data['inp1'])
    #data['inp1'] = tf.image.rot90(data['inp1'],90)
    #data['inp1'] = tf.image.transpose(data['inp1'])
    
    #data['inp1'] = random_cuout_aug(data['inp1'])
    #data['inp1'] = tf.keras.preprocessing.image.random_shear(data['inp1'],20)
    #data['inp1'] = tf.keras.preprocessing.image.random_shift(data['inp1'],0.1,0.1)
    #data['inp1'] = cutout_aug(data['inp1'])
    
    data['inp1'] = tf.image.random_hue(data['inp1'], 0.01)
    data['inp1'] = tf.image.random_saturation(data['inp1'], 0.7, 1.3)
    data['inp1'] = tf.image.random_contrast(data['inp1'], 0.8, 1.2)
    data['inp1'] = tf.image.random_brightness(data['inp1'], 0.1)
    
    
    return data, label

def get_training_dataset(filenames, labeled = True, ordered = False):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.map(data_augment, num_parallel_calls = AUTO)
    dataset = dataset.map(transform, num_parallel_calls = AUTO)
    dataset = dataset.map(norm_image, num_parallel_calls = AUTO)
    # the training dataset must repeat for several epochs
    dataset = dataset.repeat() 
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO)
    return dataset

def get_validation_dataset(filenames, labeled = True, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input1, num_parallel_calls = AUTO)
    dataset = dataset.map(norm_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # using gpu, not enought memory to use cache
    # dataset = dataset.cache()
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

def get_test_dataset(filenames, labeled = False, ordered = True):
    dataset = load_dataset(filenames, labeled = labeled, ordered = ordered)
    dataset = dataset.map(setup_input2, num_parallel_calls = AUTO)
    dataset = dataset.map(norm_image, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    # prefetch next batch while training (autotune prefetch buffer size)
    dataset = dataset.prefetch(AUTO) 
    return dataset

# function to count how many photos we have in
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

# this function parse our images and also get the target variable
def read_tfrecord_full(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "image_name": tf.io.FixedLenFeature([], tf.string), 
        "target": tf.io.FixedLenFeature([], tf.int64), 
        # meta features
        "age_approx": tf.io.FixedLenFeature([], tf.int64),
        "sex": tf.io.FixedLenFeature([], tf.int64),
        "anatom_site_general_challenge": tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    image_name = example['image_name']
    target = tf.cast(example['target'], tf.int32)
    # meta features
    data = {}
    data['age_approx'] = tf.cast(example['age_approx'], tf.int32)
    data['sex'] = tf.cast(example['sex'], tf.int32)
    data['anatom_site_general_challenge'] = tf.cast(tf.one_hot(example['anatom_site_general_challenge'], 7), tf.int32)
    return image, image_name, target, data

def load_dataset_full(filenames):        
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads = AUTO)
    # returns a dataset of (image_name, target)
    dataset = dataset.map(read_tfrecord_full, num_parallel_calls = AUTO) 
    return dataset

def get_data_full(filenames):
    dataset = load_dataset_full(filenames)
    dataset = dataset.map(setup_input3, num_parallel_calls = AUTO)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset



NUM_TRAINING_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.8)
# use validation data for training
NUM_VALIDATION_IMAGES = int(count_data_items(TRAINING_FILENAMES) * 0.2)
NUM_TEST_IMAGES = count_data_items(TEST_FILENAMES)
STEPS_PER_EPOCH = NUM_TRAINING_IMAGES // BATCH_SIZE

print('Dataset: {} training images, {} validation images, {} unlabeled test images'.format(NUM_TRAINING_IMAGES, NUM_VALIDATION_IMAGES, NUM_TEST_IMAGES))

Dataset: 36440 training images, 9110 validation images, 10982 unlabeled test images


In [22]:
_DECAY = 0.0001


class AttentionModel(tf.keras.Model):
  """Instantiates attention model.
  Uses two [kernel_size x kernel_size] convolutions and softplus as activation
  to compute an attention map with the same resolution as the featuremap.
  Features l2-normalized and aggregated using attention probabilites as weights.
  """

  def __init__(self, kernel_size=1, decay=_DECAY, name='attention'):
    """Initialization of attention model.
    Args:
      kernel_size: int, kernel size of convolutions.
      decay: float, decay for l2 regularization of kernel weights.
      name: str, name to identify model.
    """
    super(AttentionModel, self).__init__(name=name)

    # First convolutional layer (called with relu activation).
    self.conv1 = tf.keras.layers.Conv2D(
        512,
        kernel_size,
        kernel_regularizer=tf.keras.regularizers.l2(decay),
        padding='same',
        name='attn_conv1')
    self.bn_conv1 = tf.keras.layers.BatchNormalization(axis=3, name='bn_conv1')

    # Second convolutional layer, with softplus activation.
    self.conv2 = tf.keras.layers.Conv2D(
        1,
        kernel_size,
        kernel_regularizer=tf.keras.regularizers.l2(decay),
        padding='same',
        name='attn_conv2')
    self.activation_layer = tf.keras.layers.Activation('softplus')

  def call(self, inputs, training=True):
    x = self.conv1(inputs)
    x = self.bn_conv1(x, training=training)
    x = tf.nn.relu(x)

    score = self.conv2(x)
    prob = self.activation_layer(score)

    # L2-normalize the featuremap before pooling.
    inputs = tf.nn.l2_normalize(inputs, axis=-1)
    feat = tf.multiply(inputs, prob)
    # feat = tf.reduce_mean(tf.multiply(inputs, prob), [1, 2], keepdims=False)

    return feat, prob, score

In [23]:
def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed



def get_model():
    with strategy.scope():
        inp1 = tf.keras.layers.Input(shape = (img_size, img_size, 3), name = 'inp1')
        inp2 = tf.keras.layers.Input(shape = (9), name = 'inp2')
        efnetb5 = efn.EfficientNetB5(weights = 'imagenet', include_top = False)
        x = efnetb5(inp1)
        attention_model = AttentionModel(name='attention')
        x, _, _ = attention_model(x)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x1 = tf.keras.layers.Dense(256)(inp2)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        x1 = tf.keras.layers.Dropout(0.2)(x1)
        x1 = tf.keras.layers.Dense(256)(inp2)
        x1 = tf.keras.layers.Dropout(0.2)(x1)
        x1 = tf.keras.layers.Activation('relu')(x1)
        x1 = tf.keras.layers.BatchNormalization()(x1)
        concat = tf.keras.layers.concatenate([x, x1])
        #concat = tf.keras.layers.Dense(512, activation = 'relu')(concat)
        #concat = tf.keras.layers.BatchNormalization()(concat)
        #concat = tf.keras.layers.Dropout(0.2)(concat)
        #concat = tf.keras.layers.Dense(182, activation = 'relu')(concat)
        #concat = tf.keras.layers.BatchNormalization()(concat)
        #concat = tf.keras.layers.Dropout(0.2)(concat)
        output = tf.keras.layers.Dense(1, activation = 'sigmoid')(concat)

        model = tf.keras.models.Model(inputs = [inp1, inp2], outputs = [output])

        opt = tf.keras.optimizers.Adam(learning_rate = LR)
        opt = tfa.optimizers.RectifiedAdam(lr=LR)

        model.compile(
            optimizer = opt,
            # loss = [tfa.losses.SigmoidFocalCrossEntropy(gamma = 2.0, alpha = 0.80)],
            loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.05),
            metrics = [tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.AUC()]
        )

        return model
    
def train_and_predict(SUB, folds = 3):
    
    # models = []
    oof_image_name = []
    oof_target = []
    oof_prediction = []
    
    # seed everything
    seed_everything(SEED)

    kfold = KFold(folds, shuffle = True, random_state = SEED)
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(TRAINING_FILENAMES)):
        print('\n')
        print('-'*50)
        print(f'Training fold {fold + 1}')
        train_dataset = get_training_dataset([TRAINING_FILENAMES[x] for x in trn_ind], labeled = True, ordered = False)
        val_dataset = get_validation_dataset([TRAINING_FILENAMES[x] for x in val_ind], labeled = True, ordered = True)
        K.clear_session()
        model = get_model()
        # using early stopping using val loss
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', mode = 'max', patience = 5, 
                                                      verbose = 1, min_delta = 0.0001, restore_best_weights = True)
        # lr scheduler
        cb_lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_auc', factor = 0.4, patience = 2, verbose = 1, min_delta = 0.0001, mode = 'max')
        sv = tf.keras.callbacks.ModelCheckpoint(
            os.path.join(outdir,'fold-%i.h5'%fold), monitor='val_loss', verbose=0, save_best_only=True,
            save_weights_only=True, mode='min', save_freq='epoch')
    
        svauc = tf.keras.callbacks.ModelCheckpoint(
            os.path.join(outdir,'fold-%i-bestvalauc.h5'%fold), monitor='val_auc', verbose=0, save_best_only=True,
            save_weights_only=True, mode='max', save_freq='epoch')
        if fold >=3:
            history = model.fit(train_dataset, 
                                steps_per_epoch = STEPS_PER_EPOCH,
                                epochs = EPOCHS,
                                callbacks = [sv, svauc, early_stopping, cb_lr_schedule],
                                validation_data = val_dataset,
                                verbose = 2)
        model.load_weights(os.path.join(outdir,'fold-%i.h5'%fold))
        # models.append(model)
        
        # want to predict the validation set and save them for stacking
        number_of_files = count_data_items([TRAINING_FILENAMES[x] for x in val_ind])
        dataset = get_data_full([TRAINING_FILENAMES[x] for x in val_ind])
        # get the image name
        image_name = dataset.map(lambda image, image_name, target: image_name).unbatch()
        image_name = next(iter(image_name.batch(number_of_files))).numpy().astype('U')
        # get the real target
        target = dataset.map(lambda image, image_name, target: target).unbatch()
        target = next(iter(target.batch(number_of_files))).numpy()
        # predict the validation set
        image = dataset.map(lambda image, image_name, target: image)
        probabilities = model.predict(image)
        oof_image_name.extend(list(image_name))
        oof_target.extend(list(target))
        oof_prediction.extend(list(np.concatenate(probabilities)))
    
    print('\n')
    print('-'*50)
    # save oof predictions
    oof_df = pd.DataFrame({'image_name': oof_image_name, 'target': oof_target, 'predictions': oof_prediction})
    oof_df.to_csv(os.path.join(outdir,'OOF_EfficientNetB5_{}.csv'.format(img_size)), index = False)
        
    # since we are splitting the dataset and iterating separately on images and ids, order matters.
    test_ds = get_test_dataset(TEST_FILENAMES, labeled = False, ordered = True)
    test_images_ds = test_ds.map(lambda image, image_name: image)
    
    print('Computing predictions...')
    preds = []
    for i in range(folds):
        print('Loading best model...')
        model.load_weights(os.path.join(outdir,'fold-%i.h5'%i))
        preds.append([model.predict(test_images_ds)])
    probabilities = np.average(np.squeeze(np.concatenate(preds)), axis = 0)
    print('Generating submission.csv file...')
    test_ids_ds = test_ds.map(lambda image, image_name: image_name).unbatch()
    # all in one batch
    test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch
    pred_df = pd.DataFrame({'image_name': test_ids, 'target': probabilities})
    SUB.drop('target', inplace = True, axis = 1)
    SUB = SUB.merge(pred_df, on = 'image_name')
    SUB.to_csv(os.path.join(outdir,'sub_EfficientNetB5_{}.csv'.format(img_size)), index = False)
    
    return oof_target, oof_prediction
    
oof_target, oof_prediction = train_and_predict(SUB, folds=5)



--------------------------------------------------
Training fold 1


--------------------------------------------------
Training fold 2


--------------------------------------------------
Training fold 3


--------------------------------------------------
Training fold 4
Epoch 1/40
569/569 - 317s - auc: 0.7855 - loss: 0.3129 - binary_accuracy: 0.9309 - val_auc: 0.8676 - val_loss: 0.3361 - val_binary_accuracy: 0.9091 - lr: 3.0000e-04
Epoch 2/40
569/569 - 289s - auc: 0.8632 - loss: 0.2263 - binary_accuracy: 0.9570 - val_auc: 0.8917 - val_loss: 0.2821 - val_binary_accuracy: 0.9121 - lr: 3.0000e-04
Epoch 3/40
569/569 - 271s - auc: 0.8774 - loss: 0.2140 - binary_accuracy: 0.9576 - val_auc: 0.8906 - val_loss: 0.2831 - val_binary_accuracy: 0.9239 - lr: 3.0000e-04
Epoch 4/40
569/569 - 291s - auc: 0.8880 - loss: 0.2079 - binary_accuracy: 0.9598 - val_auc: 0.9028 - val_loss: 0.2682 - val_binary_accuracy: 0.9239 - lr: 3.0000e-04
Epoch 5/40
569/569 - 288s - auc: 0.8969 - loss: 0.2055 - binary_

In [24]:
# calculate our out of folds roc auc score
roc_auc = metrics.roc_auc_score(oof_target, oof_prediction)
print('Our out of folds roc auc score is: ', roc_auc)

Our out of folds roc auc score is:  0.815879943456897


In [25]:
print(outdir)

siim/res/efficientnet-res/EF5-512/all-kfold-hairaug-456-norm-meta-dense
